CLASSES

In [39]:
class Reviews:
    def __init__(self, text ,score, sentiment):
        self.text = text
        self.score = score
        self.sentiment = sentiment

    def get_sentiment(self):
        if self.score <= 2:
            return self.sentiment
        elif self.score == 3:
            return self.sentiment
        else: 
            return self.sentiment

LOAD DATA

In [40]:
import json
import random

file = r"C:\Users\fathu\Documents\Sentiment Analysis Data\database.json"
reviews = []

with open(file) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Reviews(review['reviewText'], review['overall'] ,review['sentiment']))

negative = list(filter(lambda x: x.sentiment == 'NEGATIVE', reviews))
positive = list(filter(lambda x: x.sentiment == 'POSITIVE', reviews))
data = negative + positive
random.shuffle(data)

DATA PREP

In [41]:
import re
import string

X = [x.text for x in data]

df = pd.DataFrame({'text':X})

def text_cleaning(text):
    text = text.lower()
    text = re.sub('\[.*>\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

cleaned = lambda x: text_cleaning(x)
df['cleaned_text'] = pd.DataFrame(df['text'].apply(cleaned))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

x = df['cleaned_text'].values.tolist()
y = [x.get_sentiment() for x in data]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


MODELING

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(X_train)

test_x_vectors = vectorizer.transform(X_test)

print(X_train[0])
print(train_x_vectors[0].toarray())

im not a teenager or a little kid but i love games like this you dont have to worry about running out of lives on this game its always something to do on your farm i love this one better than the other farm game im playing the only thing is you have to add people you dont know to your facebook page to have neighbors and it force closes all of a sudden now but i still play  
[[0. 0. 0. ... 0. 0. 0.]]


PIPELINE 

In [48]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

svc = svm.SVC()
pipe = Pipeline([("classifier", svc)])


params = [
    {"classifier": [svc],
    "classifier__kernel": ['linear', 'rbf'],
    "classifier__C": [1,4,8,16,32]
    },
    {"classifier": [DecisionTreeClassifier()],
    "classifier__criterion": ['gini', 'entropy'],
    "classifier__max_depth": [2,4,6,8,10,12]
    },
    {"classifier": [LogisticRegression()],
    "classifier__penalty": ['l1', 'l2'],
    "classifier__C": np.logspace(0, 4, 10)
    }
]

gridsearch = GridSearchCV(pipe, params, cv = 5, verbose=0, n_jobs=-1)
best_model = gridsearch.fit(train_x_vectors, y_train)

print(f'best models {best_model.best_estimator_}')
print("the mean accuracy:", best_model.score(test_x_vectors, y_test))
print(classification_report(y_test,best_model.best_estimator_.predict(test_x_vectors)))

<IPython.core.display.Javascript object>

C:\Users\fathu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.85762297 0.86119869 0.84031749 0.86248584 0.83416741 0.86248584
 0.82680127 0.86248584 0.82293974 0.86248584 0.6172763  0.65088663
 0.67198196 0.68349514 0.69779858 0.70580711 0.61420158 0.64151827
 0.66490292 0.67605904 0.67891881 0.69300684        nan 0.85612185
        nan 0.86105563        nan 0.85819509        nan 0.85290315
        nan 0.84796888        nan 0.84389278        nan 0.84096081
        nan 0.83995984        nan 0.83852953        nan 0.83802881]
  warnings.warn(


best models Pipeline(steps=[('classifier', SVC(C=4))])
the mean accuracy: 0.8763763763763763
              precision    recall  f1-score   support

    NEGATIVE       0.87      0.88      0.88      2962
    POSITIVE       0.88      0.87      0.88      3032

    accuracy                           0.88      5994
   macro avg       0.88      0.88      0.88      5994
weighted avg       0.88      0.88      0.88      5994

